# EDA y Tranformaciones de los data set.

Primera tranformación será pasar a formato CSV los dataset que faltan. 

In [33]:
import pandas as pd

# Lista de archivos Excel a convertir
archivos_excel = [
    "PRODUCTOS.xlsx",
    "Proveedores.xlsx",
    "CanalDeVenta.xlsx"
]

# Convertir cada archivo a CSV
for archivo in archivos_excel:
    # Cargar el archivo en un DataFrame
    df = pd.read_excel(archivo)

    # Generar el nombre del archivo CSV
    archivo_csv = archivo.replace(".xlsx", ".csv")

    # Guardar en formato CSV
    df.to_csv(archivo_csv, index=False, encoding='utf-8')

    print(f"Archivo convertido: {archivo_csv}")


FileNotFoundError: [Errno 2] No such file or directory: 'PRODUCTOS.xlsx'

## Proximo paso.

Será conocer los dataset, ver si hay errores, nulos , duplicados, outliers, para una posterior transformación de cada dataset. 

In [34]:
# Reimportar librerías después del reinicio del estado
import pandas as pd
import numpy as np

# Lista de archivos CSV
archivos_csv = {
    "Clientes": "Clientes.csv",
    "Compra": "Compra.csv",
    "Empleados": "Empleados.csv",
    "Gasto": "Gasto.csv",
    "PRODUCTOS": "PRODUCTOS.csv",
    "Proveedores": "Proveedores.csv",
    "Sucursales": "Sucursales.csv",
    "TiposDeGasto": "TiposDeGasto.csv",
    "Venta": "Venta.csv"
}

# Diccionario para almacenar los DataFrames
dataframes = {}

# Cargar y analizar cada dataset
resumen_datasets = []

for nombre, ruta in archivos_csv.items():
    try:
        # Cargar el dataset
        df = pd.read_csv(ruta)
        dataframes[nombre] = df  # Guardar en el diccionario

        # Obtener información básica
        info_basica = {
            "Dataset": nombre,
            "Filas": df.shape[0],
            "Columnas": df.shape[1],
            "Nulos (%)": df.isnull().sum().sum() / df.size * 100,
            "Duplicados": df.duplicated().sum(),
            "Outliers": np.sum((df.select_dtypes(include=[np.number]) > df.select_dtypes(include=[np.number]).quantile(0.99)).sum())
        }

        resumen_datasets.append(info_basica)
    
    except Exception as e:
        resumen_datasets.append({"Dataset": nombre, "Error": str(e)})

# Convertir en DataFrame y mostrar
df_resumen = pd.DataFrame(resumen_datasets)
df_resumen


,Dataset,Error,Filas,Columnas,Nulos (%),Duplicados,Outliers
0,Clientes,Error tokenizing data. C error: Expected 3 fie...,NaN,NaN,NaN,NaN,NaN
1,Compra,NaN,11539.0,6.0,0.000000,0.0,446.0
2,Empleados,NaN,267.0,7.0,0.000000,0.0,6.0
3,Gasto,NaN,8640.0,5.0,0.000000,0.0,174.0
4,PRODUCTOS,NaN,291.0,4.0,1.030928,0.0,3.0
5,Proveedores,NaN,14.0,7.0,2.040816,0.0,1.0
6,Sucursales,NaN,31.0,1.0,0.000000,0.0,1.0
7,TiposDeGasto,NaN,4.0,3.0,0.000000,0.0,1.0
8,Venta,NaN,46645.0,10.0,0.389324,0.0,2236.0


## Tranformaciones de datasets. 

### Transformaciones Aplicadas al Dataset "Clientes.csv"
Ahora vamos a realziar la limpieza de datos y su tranformación:

1. Tratar los valores nulos

- Rellenar valores categóricos con "el mismo nombre del registro anterior y posterior si coinciden" y valores numéricos con la mediana.

2. Corrección de tipos de datos

- identificar y convertir columnas de fecha correctamente:
    - "Fecha_Alta"
    - "Fecha_Ultima_Modificacion"

3. Normalización de textos
limpiaro espacios extras y convertido los textos a minúsculas en todas las columnas categóricas.Tambien si hubere ";" se pasará a ",". 

4. Eliminación de los datos nulos, eliminar la fila. 

5. Guardar el documento en la carpeta CSV_tranformados

#### Clientes

In [39]:
# Cargar el archivo nuevamente con el delimitador correcto
df = pd.read_csv(file_path, delimiter=";", encoding="utf-8")

# Mostrar información del dataset para verificar su correcta carga
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3407 entries, 0 to 3406
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           3407 non-null   int64  
 1   Provincia                    3376 non-null   object 
 2   Nombre_y_Apellido            3361 non-null   object 
 3   Domicilio                    3359 non-null   object 
 4   Telefono                     3317 non-null   object 
 5   Edad                         3407 non-null   int64  
 6   Localidad                    3375 non-null   object 
 7   X                            3345 non-null   object 
 8   Y                            3347 non-null   object 
 9   Fecha_Alta                   3407 non-null   object 
 10  Usuario_Alta                 3407 non-null   object 
 11  Fecha_Ultima_Modificacion    3407 non-null   object 
 12  Usuario_Ultima_Modificacion  3407 non-null   object 
 13  Marca_Baja        

(None,
    ID     Provincia        Nombre_y_Apellido  \
 0   1  Buenos Aires       HEBER JONI SANTANA   
 1   2  Buenos Aires              ANA SAPRIZA   
 2   3  Buenos Aires  FERNANDO LUIS SARALEGUI   
 3   4  Buenos Aires         MANUELA SARASOLA   
 4   5  Buenos Aires       MARIO RAÚL SARASUA   
 
                                            Domicilio  Telefono  Edad  \
 0  LAS HERAS Y BAT. 24 DE FEBRERO 4150  RINCON DE...   42-5161    58   
 1  PUEYRREDON Y DUPUY RUTA3 KM 52.500 S/N  BÂº LO...   49-7578    61   
 2                           CALDERON DE LA BARCA 498   49-3435    15   
 3                  RUTA 36 KM 45,500 S/N  EL PELIGRO   49-2883    29   
 4                     492 Y 186 S/N  COLONIA URQUIZA  491-4608    34   
 
              Localidad             X             Y  Fecha_Alta Usuario_Alta  \
 0           LOMA VERDE  -58,81850307  -34,30997088  2015-01-01        user1   
 1           SANTA ROSA  -58,73073751  -34,93908311  2015-01-01        user1   
 2               

In [40]:

# 1. Eliminar la columna "col10" que solo contiene valores nulos
df.drop(columns=["col10"], inplace=True)

# 2. Reemplazar valores nulos en categóricas con el valor anterior o posterior si coinciden
categorical_columns = ["Provincia", "Nombre_y_Apellido", "Domicilio", "Localidad"]
df[categorical_columns] = df[categorical_columns].fillna(method='ffill').fillna(method='bfill')

# 3. Reemplazar valores nulos en numéricas con la mediana
numerical_columns = ["X", "Y"]
df[numerical_columns] = df[numerical_columns].replace(',', '.', regex=True).astype(float)
df[numerical_columns] = df[numerical_columns].fillna(df[numerical_columns].median())

# 4. Convertir las columnas de fecha al formato datetime
df["Fecha_Alta"] = pd.to_datetime(df["Fecha_Alta"], errors='coerce')
df["Fecha_Ultima_Modificacion"] = pd.to_datetime(df["Fecha_Ultima_Modificacion"], errors='coerce')

# 5. Normalización de textos: convertir a minúsculas y eliminar espacios extras
for col in categorical_columns:
    df[col] = df[col].str.strip().str.lower()

# 6. Eliminar filas que aún tengan valores nulos
df.dropna(inplace=True)

# Mostrar información del dataset después de la limpieza
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 3317 entries, 0 to 3404
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           3317 non-null   int64         
 1   Provincia                    3317 non-null   object        
 2   Nombre_y_Apellido            3317 non-null   object        
 3   Domicilio                    3317 non-null   object        
 4   Telefono                     3317 non-null   object        
 5   Edad                         3317 non-null   int64         
 6   Localidad                    3317 non-null   object        
 7   X                            3317 non-null   float64       
 8   Y                            3317 non-null   float64       
 9   Fecha_Alta                   3317 non-null   datetime64[ns]
 10  Usuario_Alta                 3317 non-null   object        
 11  Fecha_Ultima_Modificacion    3317 non-null   dat

C:\Users\jerom\AppData\Local\Temp\ipykernel_18928\309919525.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[categorical_columns] = df[categorical_columns].fillna(method='ffill').fillna(method='bfill')


In [42]:
# Guardar el archivo transformado con coma como delimitador
output_path_comma = "CSV_tranformados/Clientes_transformados.csv"
df.to_csv(output_path_comma, index=False, sep=",", encoding="utf-8")

# Devolver la ruta del archivo transformado con coma
output_path_comma


'CSV_tranformados/Clientes_transformados.csv'

#### Compras

In [43]:
# Cargar el archivo y revisar su contenido inicial
file_path = "Compra.csv"

# Leer las primeras líneas para inspeccionar el formato
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()[:10]  # Leer las primeras 10 líneas para inspección

lines


['IdCompra,Fecha,IdProducto,Cantidad,Precio,IdProveedor\n',
 '1,2015-01-30,42832,13,560.51,12\n',
 '2,2015-01-30,42833,11,497.58,7\n',
 '3,2015-01-30,42834,1,588.5,6\n',
 '4,2015-01-30,42835,9,567.66,14\n',
 '5,2015-01-30,42839,14,231.31,2\n',
 '6,2015-01-30,42840,14,232.07,13\n',
 '7,2015-01-30,42841,8,236.98,4\n',
 '8,2015-01-30,42842,4,255.33,4\n',
 '9,2015-01-30,42845,5,578.61,12\n']

In [44]:
# Cargar el archivo en un DataFrame
df = pd.read_csv(file_path, delimiter=",", encoding="utf-8")

# Mostrar información del dataset antes de la limpieza
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11539 entries, 0 to 11538
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   IdCompra     11539 non-null  int64  
 1   Fecha        11539 non-null  object 
 2   IdProducto   11539 non-null  int64  
 3   Cantidad     11539 non-null  int64  
 4   Precio       11539 non-null  float64
 5   IdProveedor  11539 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 541.0+ KB


(None,
    IdCompra       Fecha  IdProducto  Cantidad  Precio  IdProveedor
 0         1  2015-01-30       42832        13  560.51           12
 1         2  2015-01-30       42833        11  497.58            7
 2         3  2015-01-30       42834         1  588.50            6
 3         4  2015-01-30       42835         9  567.66           14
 4         5  2015-01-30       42839        14  231.31            2)

In [45]:
# Convertir la columna "Fecha" a formato datetime
df["Fecha"] = pd.to_datetime(df["Fecha"], errors='coerce')

# Mostrar información después de la transformación
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11539 entries, 0 to 11538
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   IdCompra     11539 non-null  int64         
 1   Fecha        11539 non-null  datetime64[ns]
 2   IdProducto   11539 non-null  int64         
 3   Cantidad     11539 non-null  int64         
 4   Precio       11539 non-null  float64       
 5   IdProveedor  11539 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 541.0 KB


In [46]:
# Guardar el archivo transformado con coma como delimitador
output_path = "CSV_tranformados/Compra_transformada.csv"
df.to_csv(output_path, index=False, sep=",", encoding="utf-8")

# Devolver la ruta del archivo transformado
output_path


'CSV_tranformados/Compra_transformada.csv'

#### Empleados. 

In [52]:
# Cargar el archivo y revisar su contenido inicial
file_path = "Empleados.csv"

# Leer las primeras líneas para inspeccionar el formato
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()[:10]  # Leer las primeras 10 líneas para inspección

lines


['\ufeffID_empleado,Apellido,Nombre,Sucursal,Sector,Cargo,Salario\n',
 '1968,Burgos,Jeronimo,Caseros,Administración,Administrativo,32000.00\n',
 '1674,Villegas,Estefania,Caseros,Administración,Vendedor,32000.00\n',
 '1516,Fernandez,Guillermo,Caseros,Administración,Vendedor,45000.00\n',
 '1330,Ramirez,Eliana,Caseros,Administración,Vendedor,32000.00\n',
 '1657,Carmona,Jose,Caseros,Administración,Vendedor,32000.00\n',
 '1573,De santis,Marcela,Caseros,Administración,Aux. Administrativo,15000.00\n',
 '1658,Franco,Daniela,Caseros,Administración,Vendedor,32000.00\n',
 '1078,Cortes,Rafael,Caseros,Diseño,Administrativo,42000.00\n',
 '1695,Berrio,Camilo,Cabildo,Diseño,Vendedor,32000.00\n']

In [53]:
# Cargar el archivo en un DataFrame
df = pd.read_csv(file_path, delimiter=",", encoding="utf-8")

# Mostrar información del dataset antes de la limpieza
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID_empleado  267 non-null    int64  
 1   Apellido     267 non-null    object 
 2   Nombre       267 non-null    object 
 3   Sucursal     267 non-null    object 
 4   Sector       267 non-null    object 
 5   Cargo        267 non-null    object 
 6   Salario      267 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 14.7+ KB


(None,
    ID_empleado   Apellido     Nombre Sucursal          Sector           Cargo  \
 0         1968     Burgos   Jeronimo  Caseros  Administración  Administrativo   
 1         1674   Villegas  Estefania  Caseros  Administración        Vendedor   
 2         1516  Fernandez  Guillermo  Caseros  Administración        Vendedor   
 3         1330    Ramirez     Eliana  Caseros  Administración        Vendedor   
 4         1657    Carmona       Jose  Caseros  Administración        Vendedor   
 
    Salario  
 0  32000.0  
 1  32000.0  
 2  45000.0  
 3  32000.0  
 4  32000.0  )

In [54]:
# Columnas categóricas a normalizar
categorical_columns = ["Apellido", "Nombre", "Sucursal", "Sector", "Cargo"]

# Normalización de textos: convertir a minúsculas y eliminar espacios extras
for col in categorical_columns:
    df[col] = df[col].str.strip().str.lower().str.replace(";", ",")

# Mostrar información después de la transformación
df.head()


,ID_empleado,Apellido,Nombre,Sucursal,Sector,Cargo,Salario
0,1968,burgos,jeronimo,caseros,administración,administrativo,32000.0
1,1674,villegas,estefania,caseros,administración,vendedor,32000.0
2,1516,fernandez,guillermo,caseros,administración,vendedor,45000.0
3,1330,ramirez,eliana,caseros,administración,vendedor,32000.0
4,1657,carmona,jose,caseros,administración,vendedor,32000.0


In [57]:
# Guardar el archivo transformado con coma como delimitador
output_path = "CSV_tranformados/Empleados_transformados.csv"
df.to_csv(output_path, index=False, sep=",", encoding="utf-8")

# Devolver la ruta del archivo transformado
output_path


'CSV_tranformados/Empleados_transformados.csv'

#### Gasto

In [58]:
# Cargar el archivo y revisar su contenido inicial
file_path = "Gasto.csv"

# Leer las primeras líneas para inspeccionar el formato
with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()[:10]  # Leer las primeras 10 líneas para inspección

lines


['IdGasto,IdSucursal,IdTipoGasto,Fecha,Monto\n',
 '1,18,1,2015-01-01,1164.9\n',
 '2,18,2,2015-01-01,317.02\n',
 '3,18,3,2015-01-01,118.89\n',
 '4,18,4,2015-01-01,1174.23\n',
 '5,1,1,2015-01-01,1104.51\n',
 '6,1,2,2015-01-01,359.96\n',
 '7,1,3,2015-01-01,108.59\n',
 '8,1,4,2015-01-01,1029.69\n',
 '9,2,1,2015-01-01,1090.87\n']

In [59]:
# Cargar el archivo en un DataFrame
df = pd.read_csv(file_path, delimiter=",", encoding="utf-8")

# Mostrar información del dataset antes de la limpieza
df.info(), df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8640 entries, 0 to 8639
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   IdGasto      8640 non-null   int64  
 1   IdSucursal   8640 non-null   int64  
 2   IdTipoGasto  8640 non-null   int64  
 3   Fecha        8640 non-null   object 
 4   Monto        8640 non-null   float64
dtypes: float64(1), int64(3), object(1)
memory usage: 337.6+ KB


(None,
    IdGasto  IdSucursal  IdTipoGasto       Fecha    Monto
 0        1          18            1  2015-01-01  1164.90
 1        2          18            2  2015-01-01   317.02
 2        3          18            3  2015-01-01   118.89
 3        4          18            4  2015-01-01  1174.23
 4        5           1            1  2015-01-01  1104.51)

In [60]:
# Convertir la columna "Fecha" a formato datetime
df["Fecha"] = pd.to_datetime(df["Fecha"], errors='coerce')

# Mostrar información después de la transformación
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8640 entries, 0 to 8639
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   IdGasto      8640 non-null   int64         
 1   IdSucursal   8640 non-null   int64         
 2   IdTipoGasto  8640 non-null   int64         
 3   Fecha        8640 non-null   datetime64[ns]
 4   Monto        8640 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 337.6 KB


In [61]:
# Guardar el archivo transformado con coma como delimitador
output_path = "CSV_tranformados/Gasto_transformado.csv"
df.to_csv(output_path, index=False, sep=",", encoding="utf-8")

# Devolver la ruta del archivo transformado
output_path


'CSV_tranformados/Gasto_transformado.csv'